# Running attribute inference attacks on Regression Models

In this tutorial we will show how to run black-box inference attacks on regression model. This will be demonstrated on the Nursery dataset (original dataset can be found here: https://archive.ics.uci.edu/ml/datasets/nursery). 

## Preliminaries
In order to mount a successful attribute inference attack, the attacked feature must be categorical, and with a relatively small number of possible values (preferably binary).

In the case of the diabetes dataset, the sensitive feature we want to infer is the 'sex' feature, which is a binary feature.

## Load data

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

from art.utils import load_diabetes

(x_train, y_train), (x_test, y_test), _, _ = load_diabetes(test_set=0.5)

## Train MLP model

In [6]:
from sklearn.neural_network import MLPRegressor
from art.estimators.regression.scikitlearn import ScikitlearnRegressor

model = MLPRegressor(max_iter=3000)
model.fit(x_train, y_train)
art_regressor = ScikitlearnRegressor(model)

print('Base model score: ', model.score(x_test, y_test))

Base model score:  0.46101186289014806


## Attack
### Black-box attack
The black-box attack basically trains an additional classifier (called the attack model) to predict the attacked feature's value from the remaining n-1 features as well as the original (attacked) model's predictions.
#### Train attack model

In [8]:
import numpy as np
from art.attacks.inference.attribute_inference import AttributeInferenceBlackBox

attack_feature = 1  # sex

# only attacked feature
x_train_feature = x_train[:, attack_feature].copy().reshape(-1, 1)
# training data without attacked feature
x_train_for_attack = np.delete(x_train, attack_feature, 1)

bb_attack = AttributeInferenceBlackBox(art_regressor, attack_feature=attack_feature)

# get original model's predictions
x_train_predictions = np.array([np.argmax(arr) for arr in art_regressor.predict(x_train)]).reshape(-1,1)

# train attack model
bb_attack.fit(x_test)

#### Infer sensitive feature and check accuracy

In [10]:
# get inferred values
values = [-0.88085106,  1.]
inferred_train_bb = bb_attack.infer(x_train_for_attack, x_train_predictions, values=values)
# check accuracy
train_acc = np.sum(inferred_train_bb == np.around(x_train_feature, decimals=8).reshape(1,-1)) / len(inferred_train_bb)
print(train_acc)

0.7239819004524887


This means that for 72% of the training set, the attacked feature is inferred correctly using this attack.
Now let's check the precision and recall:

In [11]:
def calc_precision_recall(predicted, actual, positive_value=1):
    score = 0  # both predicted and actual are positive
    num_positive_predicted = 0  # predicted positive
    num_positive_actual = 0  # actual positive
    for i in range(len(predicted)):
        if predicted[i] == positive_value:
            num_positive_predicted += 1
        if actual[i] == positive_value:
            num_positive_actual += 1
        if predicted[i] == actual[i]:
            if predicted[i] == positive_value:
                score += 1
    
    if num_positive_predicted == 0:
        precision = 1
    else:
        precision = score / num_positive_predicted  # the fraction of predicted “Yes” responses that are correct
    if num_positive_actual == 0:
        recall = 1
    else:
        recall = score / num_positive_actual  # the fraction of “Yes” responses that are predicted correctly

    return precision, recall
    
print(calc_precision_recall(inferred_train_bb, np.around(x_train_feature, decimals=8), positive_value=1.))

(0.6470588235294118, 0.8020833333333334)


To verify the significance of these results, we now run a baseline attack that uses only the remaining features to try to predict the value of the attacked feature, with no use of the model itself.

In [13]:
from art.attacks.inference.attribute_inference import AttributeInferenceBaseline

baseline_attack = AttributeInferenceBaseline(attack_feature=attack_feature)

# train attack model
baseline_attack.fit(x_test)
# infer values
inferred_train_baseline = baseline_attack.infer(x_train_for_attack, values=values)
# check accuracy
baseline_train_acc = np.sum(inferred_train_baseline == np.around(x_train_feature, decimals=8).reshape(1,-1)) / len(inferred_train_baseline)
print(baseline_train_acc)

0.669683257918552


We can see that the black-box attack does slightly better than the baseline.